# MBAM model reduction for the MWC model of BK
Model taken from Aldrich 2002 (Equation 1) <br>
Initial parameters taken from Miranda 2013 PNAS, Suppl fig 5 <br>
Algorithm as in Transtrum 2015 (Equation 49-51) - except for signs

The code below iteratively finds limits (boundaries) and stiff nonlinear combinations of sloppy parameters for the MWC model of BK, while preserving the model output function $P_{open}(V, [Ca] \ | \ \theta)$.

The iterative model reduction proceeds as follows:
+ $\theta = \{L_0, z_L, J_0, z_J, K_d, C, D, E \}$
    * $z_L -> 0$
    
    
+ $\phi^1 = \{L_0, J_0, z_J, K_d, C, D, E \}$
    * $L_0^{1/4} \rightarrow 0, \quad C \rightarrow \infty, \quad D \rightarrow \infty, \quad E \rightarrow 0 $
    

+ $\phi^2 = \{J_0, z_J, K_d, L_0^{1/4}C, L_0^{1/4}D, L_0^{-1/4}E \}$ 
    * $L_0^{1/4}C \rightarrow 0, \quad L_0^{-1/4}E \rightarrow \infty $
    
    
+ $\phi^3 = \{J_0, z_J, K_d, CE, L_0^{1/4}D \}$ 
    * $J_0 \rightarrow 0, \quad L_0^{1/4}D \rightarrow \infty $
 
 
+ $\phi^4 = \{J_0L_0^{1/4}D, z_J, K_d, CE  \}$ 
    * $K_d \rightarrow \infty, \quad CE \rightarrow \infty $
 
 
+ $\phi^5 = \{J_0L_0^{1/4}D, z_J, CEK_d^{-1}  \}$ 

The $P_{open}(V, [Ca] \ | \ \phi^5)$ function is an excellent approximation of the original (way below experimental error), despite having only 3 parameters. Each of the parameters are meaningful, the first one describing the slope of the voltage sensitivity, the second the middle point of the voltage sensitivity and the third the calcium sensitivity.
Note that the 4 parameter-model ($\phi^4$) describes both the voltage and the calcium sensitivity curves as two-parameter sigmoids. The allosteric MWC model introduces nonlinear interaction between these two sensitivity curves.

In [2]:
using NBInclude #Includes for ipynb-s
#using JLD # Julia data I/O
nbinclude("MBAM.ipynb")
nbinclude("BK_functions/BK_functions.ipynb")


BK_cost (generic function with 1 method)

In [3]:
# Create grid to evaluate over
#=
using Iterators
Ca=([0.0, 0.7, 4.0, 12.0, 22.0, 55.0, 70.0, 95.0]*1e-6)
V=collect(-100:25:200)*1e-3
Ca = float(Ca)
V = float(V)

x_grid = Array(Any, (length(Ca)*length(V),))
iterind = 0;
for j1 in product(Ca, V)
    iterind += 1;
    x_grid[iterind] = collect(j1)
end
size(x_grid[1])
=#

(2,)

In [ ]:
#=
# Single MBAM call that runs through all iterations of BK model reduction with NON-AUTOMATED model reduction
phi0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10, 11, 12, 13, 14], 
move_dir=[-1, -1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce);
=#


  0.716084 seconds (665.10 k allocations: 31.533 MB, 0.78% gc time)
Model iteration 10, Single g_cost call takes: 
  0.060158 seconds (76.52 k allocations: 3.465 MB)
Model iteration 10, initial values:
      D = [  2.45e-05,   2.83e-04,   1.32e-03,   4.91e-02,   8.41e-02,   3.34e-01,   5.48e+00,   2.81e+01]

# Fit the reduced models to data
Using data from Aldrich 2010 Fig 4c,d fit simplified models

In [430]:
# Read the data
data_4c_control = readcsv("Figures/aldrich_fig4c_control.csv")[1:end-1,:]
data_4c_lrrc26 = readcsv("Figures/aldrich_fig4c_lrrc26.csv")
data_4d_control = readcsv("Figures/aldrich_fig4d_control.csv")
data_4d_lrrc26 = readcsv("Figures/aldrich_fig4d_lrrc26.csv");


# Models for simultaneously fitting all 4 curves

In [388]:
# join the data
data_x = [data_4c_control[:,1]; data_4c_lrrc26[:,1]; data_4d_control[:,1]; data_4d_lrrc26[:,1]]*1e-3;
data_y = [data_4c_control[:,2]; data_4c_lrrc26[:,2]; log10(data_4d_control[:,2]); log10(data_4d_lrrc26[:,2])];
data_in_log = [zeros(length(data_4c_control[:,1]),1); zeros(length(data_4c_lrrc26[:,1]),1); 
    ones(length(data_4d_control[:,1]),1); ones(length(data_4d_lrrc26[:,1]),1)].==1;
dataset_ranges = Any[1:length(data_4c_control[:,1]),
    (length(data_4c_control[:,1])+1):(length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])),
    (length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+1):(length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+length(data_4d_control[:,1])),
    (length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+length(data_4d_control[:,1])+1):length(data_x)]

# Tradeoff between log-space fit and original space fit by norm of data there
w_log = norm([data_4c_control[:,2]; data_4c_lrrc26[:,2]])./norm([data_4d_control[:,2]; data_4d_lrrc26[:,2]])

0.9766267267700984

In [395]:
# Return results in log
function to_log(x::AbstractArray, in_log::AbstractArray)
    x = copy(x);
    x[x.<1e-12]=1e-12
    x[find(a->a==true, in_log)] = log10(x[find(a->a==true, in_log)])
    return x
end

function scale_data(x::AbstractArray, to_scale::AbstractArray, w)
    x = copy(x);
    x[find(a->a==true, to_scale)] *= w;
    return x
end

scale_data (generic function with 1 method)

## Models

In [398]:
using LsqFit
using Physical

elec_resp = (Volt*(-1)*e_electron/(k_boltzmann*304*Kelvin));


function model_orig(x::AbstractArray, ϕ::AbstractArray)
    D = ϕ[1]; L0 = ϕ[2]; J0 = ϕ[3]; zJ = ϕ[4]; zL = ϕ[5];
    Popen = (1+
    (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
    ./
    ( L0.*exp(-zL .* x .* elec_resp ).*(1+D.*J0.*exp(-zJ .* x .* elec_resp )).^4)
    ).^-1
    
    logPopen = copy(Popen);
    for p1 = 1:length(logPopen)
        logPopen[p1] = Popen[p1]>0?log(Popen[p1]):-6
    end
    return Popen
end

function model_red(x::AbstractArray, ϕ::AbstractArray)
    DL0 = ϕ[1]; J0 = ϕ[2]; zJ = ϕ[3]; zL = ϕ[4];
    Popen = (1+
    (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
    ./
    (exp(-zL .* x .* elec_resp ).*(DL0.*J0.*exp(-zJ .* x .* elec_resp )).^4)
    ).^-1
    return Popen
end

function model_Dmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_orig(x[data_ranges[1]], [ϕ[1]; ϕ[2:5]])
    y_out[data_ranges[2]] = model_orig(x[data_ranges[2]], [ϕ[6]; ϕ[2:5]])
    y_out[data_ranges[3]] = model_orig(x[data_ranges[3]], [ϕ[1]; ϕ[2:5]])
    y_out[data_ranges[4]] = model_orig(x[data_ranges[4]], [ϕ[6]; ϕ[2:5]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end


function model_LDmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_red(x[data_ranges[1]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[2]] = model_red(x[data_ranges[2]], [ϕ[5]; ϕ[2:4]])
    y_out[data_ranges[3]] = model_red(x[data_ranges[3]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[4]] = model_red(x[data_ranges[4]], [ϕ[5]; ϕ[2:4]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end

model_LDmoves (generic function with 1 method)

In [423]:
weight_of_log_fits = 0;

fit_orig = LsqFit.curve_fit((a,b)->model_Dmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
                                data_x, scale_data(data_y, data_in_log, weight_of_log_fits), [phi0, 412]);

fit_red = LsqFit.curve_fit((a,b)->model_LDmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
                                data_x, scale_data(data_y, data_in_log, weight_of_log_fits), [0.91, phi0[3:5], 3]);

In [424]:
y_pred_orig = model_Dmoves(data_x, [phi0, 412], data_ranges=dataset_ranges, in_log=data_in_log);
y_pred_origfit = model_Dmoves(data_x, fit_orig.param, data_ranges=dataset_ranges, in_log=data_in_log);
y_pred_redfit = model_LDmoves(data_x, fit_red.param, data_ranges=dataset_ranges, in_log=data_in_log);

In [425]:
y_pred_origfit

70-element Array{Float64,1}:
  1.08157e-5 
  2.07252e-5 
  4.33877e-5 
  0.000100576
  0.000719938
  0.00213798 
  0.00632252 
  0.0184487  
  0.0521389  
  0.128564   
  0.267051   
  0.454124   
  0.63027    
  ⋮          
 -0.404363   
 -0.14501    
 -0.0411752  
 -0.0117154  
 -0.00328744 
 -0.00103613 
 -0.000343328
 -0.000131744
 -5.6562e-5  
 -2.65181e-5 
 -1.3976e-5  
 -8.33136e-6 

In [426]:


p1 = Plot([scatter(;x=data_x[dataset_ranges[3]], y=data_y[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=data_y[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_origfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_origfit[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_redfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_redfit[dataset_ranges[4]])
    ]
        )

p2 = Plot([scatter(;x=data_x[dataset_ranges[1]], y=data_y[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=data_y[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_origfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_origfit[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_redfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_redfit[dataset_ranges[2]])
    ]
        )

plt = plot([p1
    p2])

In [427]:
PlotlyJS.relayout!(plt, height=1000)

In [428]:
fit_red.param

5-element Array{Float64,1}:
  1.13199  
  0.0975634
  0.429144 
  0.254361 
 10.1879   

In [429]:
fit_orig.param

6-element Array{Float64,1}:
  27.927     
   4.34291e-6
   0.07139   
   0.454448  
   0.202457  
 283.174     